In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [6]:
class Residual(nn.Module):
    def __init__(self, input_channels, num_channels, use_1x1conv=False, strides=1):
        super(Residual, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)
    
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        else:
            X = self.conv1(X)
        Y += X
        return F.relu(Y)

In [7]:
from torch.utils.tensorboard import SummaryWriter
import os
import torch
os.makedirs(r"/home/cxmd/文档/git_files/neural_network/ResNet/run1", exist_ok=True)
resnet = Residual(input_channels=3, num_channels=10)
writer = SummaryWriter('/home/cxmd/文档/git_files/neural_network/ResNet/run1')
writer.add_graph(resnet, torch.randn(1, 3, 32, 32))
writer.close()